# 🤖 RAG-Powered Personal Chatbot  
### *A Mini Project by Mehdy Mokhtari*  

This notebook demonstrates the creation of a **Retrieval-Augmented Generation (RAG)** chatbot capable of answering questions based on **personal or private documents**.  
The project leverages **LangChain**, **FAISS**, **OpenAI LLMs**, and **Python** to build an intelligent assistant that retrieves relevant information and generates contextual responses.

---

**📘 Project Highlights:**
- 🧠 Uses **RAG architecture** (Retriever + Generator) for accurate, data-grounded answers.  
- 🔍 Employs **FAISS** for efficient vector search and similarity matching.  
- 📄 Supports **custom document ingestion** (PDFs, text files, etc.).  
- ⚙️ Built with **LangChain** for modular and scalable AI pipelines.  
- 💬 Optional **Streamlit interface** for interactive chatting.

---

> Developed by *Mehdy Mokhtari* | 2025  
> “Empowering data-driven conversations through Retrieval-Augmented Generation.”


In [5]:
# import shutil
# import os

# # Replace 'your_directory_path' with the path to the directory you want to delete
# shutil.rmtree(os.path.join(".", "data"))


In [6]:
# !pip install langchain faiss-cpu huggingface_hub langchain_community hazm sentence-transformers openai
# 

In [7]:
# !pip install hazm
# 

## 1. DataLoader — PDF Downloader, Text Extractor & Caching

This script automates the process of:
- **Downloading** PDF files from URLs listed in `links.txt` using the `requests` library.  
- **Extracting text** from each PDF with `PyPDFLoader` (from `langchain_community.document_loaders`).  
- **Caching extracted text** per file in the `cached_extracted_data/` folder as JSONL files, so future runs skip re-downloading and re-processing.  
- **Auto-refreshing cache** only if a PDF changes or is newly added.


In [1]:
import os
import json
import requests
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document

TXT_PATH = "links.txt"
RAW_DIR = os.path.join(".", "data")
CACHE_DIR = os.path.join(".", "cached_extracted_data")

os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(CACHE_DIR, exist_ok=True)

# 1) Read links
with open(TXT_PATH, encoding="utf-8") as f:
    urls = [line.strip() for line in f if line.strip()]

docs = []

# 2) Download PDFs only if data folder is empty
if not os.listdir(RAW_DIR):
    print("Data folder is empty — downloading all PDFs.")
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/122.0 Safari/537.36"
        )
    }

    for idx, url in enumerate(urls, start=1):
        file_name = f"doc_{idx}.pdf"  # predictable names to avoid .aspx confusion
        pdf_path = os.path.join(RAW_DIR, file_name)

        try:
            resp = requests.get(url, headers=headers, stream=True, allow_redirects=True, timeout=60)
            resp.raise_for_status()

            # Only save if server actually returns a PDF
            ctype = resp.headers.get("Content-Type", "")
            if "application/pdf" in ctype.lower():
                with open(pdf_path, "wb") as f:
                    for chunk in resp.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                print(f"Downloaded {pdf_path}")
            else:
                print(f"⚠️ Skipped: URL did not return a PDF (Content-Type={ctype}) -> {url}")
        except requests.RequestException as e:
            print(f"❌ Error downloading {url}: {e}")
else:
    print("Data folder already contains files — using cached PDFs if available.")

def cache_path_for(pdf_filename: str) -> str:
    """Return the per-PDF cache file path (JSONL) based on the PDF file name."""
    base, _ = os.path.splitext(pdf_filename)
    return os.path.join(CACHE_DIR, f"{base}.jsonl")

def write_cache(jsonl_path: str, page_docs):
    """Write a list of LangChain Documents (or dicts) to JSONL cache."""
    with open(jsonl_path, "w", encoding="utf-8") as f:
        for d in page_docs:
            if Document and isinstance(d, Document):
                record = {"page_content": d.page_content, "metadata": d.metadata or {}}
            else:
                # assume dict-like
                record = {
                    "page_content": d.get("page_content", ""),
                    "metadata": d.get("metadata", {}),
                }
            f.write(json.dumps(record, ensure_ascii=False) + "\n")

def read_cache(jsonl_path: str):
    """Read JSONL cache back into a list of Documents (if available) or dicts."""
    loaded = []
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            if Document:
                loaded.append(Document(page_content=obj["page_content"], metadata=obj.get("metadata", {})))
            else:
                loaded.append({"page_content": obj["page_content"], "metadata": obj.get("metadata", {})})
    return loaded

# 3) Load all PDFs from RAW_DIR, caching extracted text per file
for file_name in sorted(os.listdir(RAW_DIR)):
    if not file_name.lower().endswith(".pdf"):
        continue

    pdf_path = os.path.join(RAW_DIR, file_name)
    jsonl_cache = cache_path_for(file_name)

    # If cache exists and is up-to-date compared to the PDF, load from cache
    use_cache = False
    if os.path.exists(jsonl_cache):
        try:
            pdf_mtime = os.path.getmtime(pdf_path)
            cache_mtime = os.path.getmtime(jsonl_cache)
            use_cache = cache_mtime >= pdf_mtime
        except OSError:
            use_cache = False

    if use_cache:
        page_docs = read_cache(jsonl_cache)
        docs.extend(page_docs)
        print(f"Loaded from cache: {jsonl_cache}")
        continue

    # Otherwise, parse PDF, then write cache
    try:
        loader = PyPDFLoader(pdf_path)
        page_docs = loader.load()  # list of per-page Documents
        docs.extend(page_docs)
        write_cache(jsonl_cache, page_docs)
        print(f"Parsed & cached: {pdf_path} -> {jsonl_cache}")
    except Exception as e:
        print(f"❌ Error parsing {pdf_path}: {e}")

print(f"Total documents loaded: {len(docs)}")


Data folder already contains files — using cached PDFs if available.
Loaded from cache: .\cached_extracted_data\doc_1.jsonl
Loaded from cache: .\cached_extracted_data\doc_2.jsonl
Loaded from cache: .\cached_extracted_data\doc_3.jsonl
Loaded from cache: .\cached_extracted_data\doc_4.jsonl
Loaded from cache: .\cached_extracted_data\doc_5.jsonl
Loaded from cache: .\cached_extracted_data\doc_6.jsonl
Loaded from cache: .\cached_extracted_data\doc_7.jsonl
Loaded from cache: .\cached_extracted_data\doc_8.jsonl
Total documents loaded: 23


In [2]:
# docs[0]
docs[7]

Document(metadata={'producer': 'Microsoft® Word LTSC', 'creator': 'Microsoft® Word LTSC', 'creationdate': '2024-05-14T12:02:42+03:30', 'moddate': '2024-05-14T12:03:24+03:30', 'author': 'Pishgam Rayaneh', 'source': '.\\data\\doc_3.pdf', 'total_pages': 5, 'page': 3, 'page_label': '4'}, page_content='3 \n \nماده6-درخصدص عهم بهرمنهی متقاض   و افراد تحت تکفل آنان در زماان اخاذ تساهيلات از زماين مساکدن  ياا واحاه \nمسکدن ، بانکهای عامل به سامانه جامع طرحهای حمايت  وزارت را  و شهرسازی به نشان  Temgt.mrud.ir مراجعاه و\nدر صفحه اصل  در قسمت "استعلام مالکيت" با درج شماره کدملی متقاضیان نسبت استعلام وضعيت ايشان اقهام نماينه. \nماده 7- حهاقل مستنهات و مهارک ذيل جهت پرداخت تسهيلات يادشه  از متقاضيان اخذ شدد: \n1- تسهيلات وديعه: لزوم ارائه قرارداد اجار  با درج که رهگيری در آن. \n2-  تسهيلات  خريه:  لزوم ارائه مبايعهنامه  با درج که رهگيری  در آن و اعطاء تسهيلات  همزمان با انتقال سنه رسم  ملک به \nنام تسهيلاتگيرنه .  \n3-  تسهيلات ساخت: لزوم ارائه پروانه ساخت معتبر و ارائه اسناد مبين مالکيت در مداردی 

## 2. FAISS Vector Database & Retriever

This step processes the documents stored in memory (`docs`) by:
- **Normalizing Persian text** using the `hazm` library (fixing Arabic/Persian character differences and removing unnecessary characters).
- **Chunking** long documents into smaller segments using `RecursiveCharacterTextSplitter`, with Persian-aware separators.
- **Generating embeddings** using `HuggingFaceEmbeddings` (or optionally OpenAI embeddings).
- **Building a FAISS vector database** to store document chunks and make them searchable.
- **Persisting the FAISS index** to disk for future use.
- **Creating a retriever** using the FAISS vector store to fetch relevant document chunks based on queries.

This setup enables efficient retrieval of contextually relevant document parts to power the RAG chatbot.


In [10]:
# %pip install --upgrade --force-reinstall --no-cache-dir -r requirements.txt


In [3]:
import sys, site, platform
print("PY:", sys.executable)
print("VER:", sys.version)
print("SITE:", site.getsitepackages())


PY: c:\Users\NoteBook\Desktop\programing\Gen AI\GenAI-course\CA_04_RAG_Chain_Personal_Chatbot\.venv\Scripts\python.exe
VER: 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
SITE: ['c:\\Users\\NoteBook\\Desktop\\programing\\Gen AI\\GenAI-course\\CA_04_RAG_Chain_Personal_Chatbot\\.venv', 'c:\\Users\\NoteBook\\Desktop\\programing\\Gen AI\\GenAI-course\\CA_04_RAG_Chain_Personal_Chatbot\\.venv\\Lib\\site-packages']


In [4]:
import sys, nltk
print("Python:", sys.version)
print("nltk:", nltk.__version__)
# print("hazm:", hazm.__version__)


Python: 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
nltk: 3.9.2


In [5]:
#  FAISS - VecDB

import os
import pickle
from typing import List
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer
from langchain_community.embeddings import HuggingFaceEmbeddings
import hazm

INDEX_DIR = os.path.join(".", "vectorstores", "faiss_simple")
os.makedirs(INDEX_DIR, exist_ok=True)
RETRIEVER_FILE = os.path.join(INDEX_DIR, "retriever.pkl")

c:\Users\NoteBook\Desktop\programing\Gen AI\GenAI-course\CA_04_RAG_Chain_Personal_Chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Chunking for Persian - token aware (not char)

MODEL_NAME = "xmanii/maux-gte-persian"

_tok = AutoTokenizer.from_pretrained(MODEL_NAME)
def token_len(s: str) -> int:
    return len(_tok.encode(s, add_special_tokens=False))

# derive safe chunk/overlap from tokenizer max length to avoid truncation
_max_len = getattr(_tok, "model_max_length", 512) or 512
_safe_chunk = min(480, _max_len - 32)   
_safe_overlap = max(64, int(_safe_chunk * 0.15))

print("_safe_chunk", _safe_chunk)
print("_safe_overlap", _safe_overlap)

text_splitter = RecursiveCharacterTextSplitter(
    separators=[
        "\n\n", "\n", "۔", ".", "؟", "!", "؛", ":", "،", "٬", " ", ""
    ],
    chunk_size=_safe_chunk,
    chunk_overlap=_safe_overlap,
    length_function=token_len,  # token-aware chunks → less truncation, better retrieval
    is_separator_regex=False,
)


_safe_chunk 480
_safe_overlap 72


In [7]:

# Embedding

embeddings = HuggingFaceEmbeddings(
    model_name=MODEL_NAME,
    encode_kwargs={"normalize_embeddings": True},
    model_kwargs={"trust_remote_code": True}
)

# Option 1 – balanced (Persian + English)
# embeddings = HuggingFaceEmbeddings(
#     model_name="heydariAI/persian-embeddings",
#     encode_kwargs={"normalize_embeddings": True}
# )

# Option 2 – optimized for Persian
# embeddings = HuggingFaceEmbeddings(
#     model_name="xmanii/maux-gte-persian",
#     encode_kwargs={"normalize_embeddings": True}
# )

# Option 3 – best performance (Hakim)
# embeddings = HuggingFaceEmbeddings(
#     model_name="MCINext/Hakim",
#     encode_kwargs={"normalize_embeddings": True}
# )



C:\Users\NoteBook\AppData\Local\Temp\ipykernel_9948\3518595760.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(
You try to use a model that was created with version 3.2.0, however, your version is 2.6.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [8]:
from statistics import mean

# use the same tokenizer you already loaded
def count_tokens(text: str) -> int:
    return len(_tok.encode(text, add_special_tokens=False))

token_counts = [count_tokens(d.page_content) for d in docs]
avg_tokens = mean(token_counts)
total_tokens = sum(token_counts)

print(f"📄 Total documents: {len(docs)}")
print(f"🔢 Total tokens: {total_tokens:,}")
print(f"📊 Average tokens per document: {avg_tokens:.1f}")
print(f"📈 Min: {min(token_counts)}, Max: {max(token_counts)}")


📄 Total documents: 23
🔢 Total tokens: 9,557
📊 Average tokens per document: 415.5
📈 Min: 72, Max: 861


In [9]:
_normalizer = hazm.Normalizer(persian_numbers=True, persian_style=True)

def normalize_fa(text: str) -> str:
    if not text:
        return text
    t = _normalizer.normalize(text)
    t = t.replace("\ufeff", "") 
    t = t.replace("ي", "ی").replace("ك", "ک")
    t = t.replace("ـ", "").replace("\u200d", "") 
    return " ".join(t.split()) 

def normalize_docs(docs: List[Document]) -> List[Document]:
    out = []
    for d in docs:
        pc = normalize_fa(d.page_content)
        out.append(Document(page_content=pc, metadata=dict(d.metadata or {})))
    return out

normalized_docs = normalize_docs(docs)

# Chunk long docs
chunked_docs = text_splitter.split_documents(normalized_docs)

# Check if FAISS index exists, if so, load it; otherwise, create it
if os.path.exists(os.path.join(INDEX_DIR, "faiss.index")):
    print("Loading existing FAISS index...")
    vectorstore = FAISS.load_local(INDEX_DIR, embeddings)
else:
    print("Creating new FAISS index...")
    vectorstore = FAISS.from_documents(chunked_docs, embeddings)
    vectorstore.save_local(INDEX_DIR)  # Save the newly created index

# Load or create the retriever
def save_retriever(retriever, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(retriever, f)

def load_retriever(file_path):
    with open(file_path, 'rb') as f:
        return pickle.load(f)

# Save retriever to file
if os.path.exists(RETRIEVER_FILE):
    print("✅ Loading saved retriever...")
    retriever_faiss = load_retriever(RETRIEVER_FILE)
else:
    print("✅ Creating new retriever...")
    retriever_faiss = vectorstore.as_retriever(
        search_type="mmr",  # or similarity
        search_kwargs={"k": 5, "fetch_k": 20, "lambda_mult": 0.5}
    )
    save_retriever(retriever_faiss, RETRIEVER_FILE)  # Save retriever to file

# Print confirmation
print(f"✅ FAISS index built at: {INDEX_DIR}")
print(f"✅ Chunks indexed: {vectorstore.index.ntotal}")
print("✅ Retriever ready: retriever.get_relevant_documents(<query>)")

Creating new FAISS index...
✅ Creating new retriever...
✅ FAISS index built at: .\vectorstores\faiss_simple
✅ Chunks indexed: 38
✅ Retriever ready: retriever.get_relevant_documents(<query>)


In [10]:
results = retriever_faiss.get_relevant_documents("میتوانی کمی راجب تسهیلات فرزندآوری توضیح بدی.")
for r in results:
    print(r.metadata.get("source"), r.metadata.get("page"), r.page_content[:120], "...")

.\data\doc_3.pdf 2 . ماده ۴- احراز ولادت فرزنه متقاضیان دریافت تسهیلات مدضدع قاندن، بایه تدسط بانک عامل ۱- با ملاحظه اصل شناسنامه زوج، زوجه ...
.\data\doc_8.pdf 0 . ir مدیران عامل محترم بانکهای ملی ا یران، بانک سپه، تجارت، رفاه کارگران، صادرات ایران، کشاورزی، ملت، مسکن، توسعه تعاون، ...
.\data\doc_7.pdf 0 . دم اجرای هریک از بندهای این تبصره (مشتمل بر تأخیر در پرداخت این تسهیلات یا دریافت ضمانت فراتر از حدود این قانون) تالف  ...
.\data\doc_8.pdf 0 ؛ تسهتلات قرضالحسنه ا دواج، تسهتلات قرضالحسنه فر ندآوری، تسهتلات قرضالحسنه ودییته یتا اریتد یتا ساات مسکر به شرح یر اقدا ...
.\data\doc_1.pdf 0 تهران– بلوار میرداماد – پلاک ۱۹۸ تلفن۲۹۹۵۱ کدپستی۳۳۱۱۱-۱۵۴۹۶ فاکس:۶۶۷۳۵۶۷۴ سایت اینترنتیwww. cbi. ir مدیران عامل محترم ب ...


C:\Users\NoteBook\AppData\Local\Temp\ipykernel_9948\3306283593.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  results = retriever_faiss.get_relevant_documents("میتوانی کمی راجب تسهیلات فرزندآوری توضیح بدی.")


## Parent-Document Retriever (Chroma + Local DocStore) - method 2

Implements a hierarchical RAG pipeline for Persian text using **ParentDocumentRetriever**.  
Key steps:
- **Text Normalization (Hazm)** for Persian script cleanup.  
- **Parent–Child Chunking** with `RecursiveCharacterTextSplitter`.  
- **Embeddings:** Persian HuggingFace model for semantic vectors.  
- **Vector Store:** `Chroma` for child chunk embeddings.  
- **DocStore:** Custom `PickleDocStore` (extends `BaseStore`) to persist parent documents locally.  
- **Retriever:** MMR-based search for relevant, diverse document retrieval.  
- **Persistence:** Saves both retriever (`retriever.pkl`) and vector index for reuse.


In [ ]:
# # --- RAG Option 2: Parent-Document Retriever (Chroma + Local DocStore, Persian-aware) ---
# import os
# import uuid
# from typing import List
# from langchain.docstore.document import Document
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.retrievers import ParentDocumentRetriever
# from langchain_community.vectorstores import Chroma
# from langchain.storage import LocalFileStore
# from langchain_community.embeddings import HuggingFaceEmbeddings
# import hazm
# from transformers import AutoTokenizer

# INDEX_DIR = os.path.join(".", "vectorstores", "parent_chroma")
# PARENT_STORE_DIR = os.path.join(".", "parent_store")
# RETRIEVER_FILE = os.path.join(INDEX_DIR, "retriever.pkl")
# os.makedirs(INDEX_DIR, exist_ok=True)
# os.makedirs(PARENT_STORE_DIR, exist_ok=True)

In [ ]:
# _normalizer = hazm.Normalizer(persian_numbers=True, persian_style=True)

# def normalize_fa(text: str) -> str:
#     if not text:
#         return text
#     t = _normalizer.normalize(text)
#     t = t.replace("ي", "ی").replace("ك", "ک")  # Arabic -> Persian forms
#     t = t.replace("ـ", "").replace("\u200d", "")  # remove tatweel & ZWJ
#     return " ".join(t.split())  # Collapse extra whitespaces

# def normalize_docs(docs: List[Document]) -> List[Document]:
#     out = []
#     for d in docs:
#         normalized_content = normalize_fa(d.page_content)
#         out.append(Document(page_content=normalized_content, metadata=dict(d.metadata or {})))
#     return out

# normalized_docs = normalize_docs(docs)  # <-- your in-memory docs

# # ---- Splitters (Parent: bigger, Child: smaller) ----
# parent_splitter = RecursiveCharacterTextSplitter(
#     separators=["\n\n", "\n", "۔", ".", "؟", "!", "؛", ":", "،", "٬", "\u200c", " ", ""],
#     chunk_size=1500,
#     chunk_overlap=150,
#     is_separator_regex=False,
# )

# child_splitter = RecursiveCharacterTextSplitter(
#     separators=["\n\n", "\n", "۔", ".", "؟", "!", "؛", ":", "،", "٬", "\u200c", " ", ""],
#     chunk_size=400,
#     chunk_overlap=100,
#     is_separator_regex=False,
# )

# # ---- Vector store (child vectors) ----
# vectorstore = Chroma(
#     collection_name="parent_child_index",
#     embedding_function=embeddings,
#     persist_directory=INDEX_DIR,
# )

# # ---- Parent docstore ----
# # ---- Parent docstore ----
# # Adapter: serialize Document <-> bytes over LocalFileStore, AND subclass BaseStore
# import pickle
# from typing import List, Tuple, Optional, Iterator
# from langchain.storage import LocalFileStore, BaseStore
# from langchain.docstore.document import Document

# class PickleDocStore(BaseStore[str, Document]):
#     def __init__(self, base_store: LocalFileStore):
#         self._base = base_store  # stores raw bytes on disk

#     # List[(key, Document)] -> bytes
#     def mset(self, key_value_pairs: List[Tuple[str, Document]]) -> None:
#         to_bytes = [(k, pickle.dumps(v)) for k, v in key_value_pairs]
#         self._base.mset(to_bytes)

#     # List[key] -> List[Optional[Document]]
#     def mget(self, keys: List[str]) -> List[Optional[Document]]:
#         raw = self._base.mget(keys)
#         return [pickle.loads(b) if b is not None else None for b in raw]

#     # delete by keys
#     def mdelete(self, keys: List[str]) -> None:
#         self._base.mdelete(keys)

#     # iterate keys
#     def yield_keys(self) -> Iterator[str]:
#         yield from self._base.yield_keys()

# # instantiate the byte store + wrapped BaseStore
# _byte_store = LocalFileStore(PARENT_STORE_DIR)
# docstore = PickleDocStore(_byte_store)



# # ---- Build Parent-Document Retriever ----
# retriever = ParentDocumentRetriever(
#     vectorstore=vectorstore,
#     docstore=docstore,
#     child_splitter=child_splitter,
#     parent_splitter=parent_splitter,
#     search_type="mmr",
#     search_kwargs={"k": 5, "fetch_k": 20, "lambda_mult": 0.5},
# )

# # Stable IDs per parent doc (use metadata if available, else UUID)
# def doc_id(d: Document, i: int) -> str:
#     src = (d.metadata or {}).get("source")
#     return f"{src}::p{i}" if src else f"doc-{i}-{uuid.uuid4().hex[:8]}"

# # Use ALL normalized docs as parents
# parent_docs = normalized_docs  # <-- essential change (no filtering)
# ids = [doc_id(d, i) for i, d in enumerate(parent_docs)]  # (ok to keep even if not used)

# # ---- Save/Load retriever functions ----
# def save_retriever(retriever, file_path):
#     with open(file_path, "wb") as f:
#         pickle.dump(retriever, f)

# def load_retriever(file_path):
#     with open(file_path, "rb") as f:
#         return pickle.load(f)

# # ---- Check if retriever exists, otherwise create and save it ----
# if os.path.exists(RETRIEVER_FILE):
#     print("Loading saved retriever...")
#     retriever = load_retriever(RETRIEVER_FILE)
# else:
#     print("Creating new retriever...")
#     retriever.add_documents(parent_docs)  # parents → split to children internally
#     save_retriever(retriever, RETRIEVER_FILE)

# # Persist child vectors on disk
# vectorstore.persist()

# # ---- Debug and Status Output ----
# print("Parent-Document retriever ready.")
# print(f"Child vectors: {vectorstore._collection.count()}")
# print(f"Parent store dir: {PARENT_STORE_DIR}")


In [ ]:
# query = "میتوانی کمی راجب تسهیلات فرزندآوری توضیح بدی."
# results = retriever.get_relevant_documents(query)
# for r in results:
#     print(r.metadata.get("source"), "—", r.page_content[:120], "…")

## Hybrid Persian Document Retrieval System (TF-IDF + Weaviate)

This notebook demonstrates a **hybrid information retrieval pipeline** for **Persian text documents**, integrating both **sparse (TF-IDF)** and **dense (embedding-based)** search mechanisms using **LangChain**, **Weaviate**, and **Hugging Face embeddings**.

Key highlights:
- **Persian text normalization** via `hazm` for consistent linguistic preprocessing.  
- **Hierarchical document chunking** using `RecursiveCharacterTextSplitter` to optimize retrieval granularity.  
- **Hybrid retriever architecture** combining `TFIDFRetriever` and `Weaviate` vector store through `EnsembleRetriever`.  
- **Persistence support** for reusing fitted vectorizers and retriever configurations using `joblib`.  
- **Fully compatible with LangChain’s retriever interface** for seamless integration into downstream QA or RAG pipelines.  

This implementation reflects a production-ready approach to **hybrid semantic search** for Persian-language corpora.


In [ ]:
# import os
# import uuid
# from typing import List
# import weaviate
# import hazm
# from transformers import AutoTokenizer
# from sklearn.feature_extraction.text import TfidfVectorizer
# import joblib
# from langchain.docstore.document import Document
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.vectorstores import Weaviate
# from langchain_community.retrievers import TFIDFRetriever
# from langchain.retrievers import EnsembleRetriever

# INDEX_DIR = os.path.join(".", "vectorstores", "weaviate")
# RETRIEVER_FILE = os.path.join(INDEX_DIR, "hybrid_retriever.joblib")
# os.makedirs(INDEX_DIR, exist_ok=True)

In [ ]:
# client = weaviate.Client("http://localhost:8080")

# print("weaviate ready:", client.is_ready())
# print("schema:", client.schema.get())

weaviate ready: True
schema: {'classes': [{'class': 'Persian_documents_index', 'description': "This property was generated by Weaviate's auto-schema feature on Fri Nov  7 18:34:19 2025", 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2}, 'cleanupIntervalSeconds': 60, 'stopwords': {'additions': None, 'preset': 'en', 'removals': None}}, 'multiTenancyConfig': {'enabled': False}, 'properties': [{'dataType': ['text'], 'description': "This property was generated by Weaviate's auto-schema feature on Fri Nov  7 18:34:19 2025", 'indexFilterable': True, 'indexSearchable': True, 'name': 'source', 'tokenization': 'word'}, {'dataType': ['number'], 'description': "This property was generated by Weaviate's auto-schema feature on Fri Nov  7 18:34:19 2025", 'indexFilterable': True, 'indexSearchable': False, 'name': 'total_pages'}, {'dataType': ['text'], 'description': "This property was generated by Weaviate's auto-schema feature on Fri Nov  7 18:34:19 2025", 'indexFilterable': True, 'indexSearcha

In [ ]:
# TFIDF_PARAMS = {"max_features": 5000, "ngram_range": (1, 3)}

# # ---- Persian normalization ----
# _normalizer = hazm.Normalizer(persian_numbers=True, persian_style=True)
# def normalize_fa(text: str) -> str:
#     if not text:
#         return text
#     t = _normalizer.normalize(text)
#     t = t.replace("ي", "ی").replace("ك", "ک")       # Arabic -> Persian forms
#     t = t.replace("ـ", "").replace("\u200d", "")     # remove tatweel & ZWJ (keep ZWNJ)
#     return " ".join(t.split())                       # collapse extra whitespaces

# def normalize_docs(docs: List[Document]) -> List[Document]:
#     out = []
#     for d in docs:
#         out.append(Document(page_content=normalize_fa(d.page_content),
#                             metadata=dict(d.metadata or {})))
#     return out

# # ---- splitters (parent / child) ----
# parent_splitter = RecursiveCharacterTextSplitter(
#     separators=["\n\n", "\n", "۔", ".", "؟", "!", "؛", ":", "،", "٬", "\u200c", " ", ""],
#     chunk_size=1500,
#     chunk_overlap=150,
#     is_separator_regex=False,
# )
# child_splitter = RecursiveCharacterTextSplitter(
#     separators=["\n\n", "\n", "۔", ".", "؟", "!", "؛", ":", "،", "٬", "\u200c", " ", ""],
#     chunk_size=400,
#     chunk_overlap=100,
#     is_separator_regex=False,
# )

# # ---- chunking ----
# def chunk_documents(docs: List[Document]) -> List[Document]:
#     chunks = []
#     for d in docs:
#         parents = parent_splitter.split_documents([d])
#         for p in parents:
#             chunks.extend(child_splitter.split_documents([p]))
#     return chunks

# # 1) normalize
# normalized_docs = normalize_docs(docs)          # <-- 'docs' must exist in your notebook
# # 2) chunk
# chunked_docs = chunk_documents(normalized_docs)
# print(f"docs: {len(docs)}  →  chunks: {len(chunked_docs)}")

# # ---- Weaviate VectorStore (dense) ----
# dense_vs = Weaviate(
#     client=client,
#     index_name="persian_documents_index",
#     text_key="text",          # where chunk text will be stored in Weaviate objects
#     embedding=embeddings,     # embedding function (already created in your code)
#     by_text=False 
# )

# # ---- save/load helpers (persist TF-IDF + ensemble weights only) ----
# def save_retriever(hybrid: EnsembleRetriever, file_path: str) -> None:
#     tfidf_vec = hybrid.retrievers[0].vectorizer  # fitted TfidfVectorizer (first retriever is TF-IDF)
#     data = {
#         "weights": hybrid.weights,
#         "tfidf_vectorizer": tfidf_vec,           # save fitted vectorizer directly
#         "index_name": "persian_documents_index",
#         "text_key": "text",
#     }
#     joblib.dump(data, file_path)

# def load_retriever(file_path: str, chunked: List[Document]) -> EnsembleRetriever:
#     data = joblib.load(file_path)

#     # rebuild TF-IDF retriever with loaded fitted vectorizer
#     vec: TfidfVectorizer = data["tfidf_vectorizer"]
#     texts = [d.page_content for d in chunked]
#     tfidf_array = vec.transform(texts)  # transform (vectorizer already fitted)
#     tfidf_docs = [Document(page_content=t, metadata=chunked[i].metadata) for i, t in enumerate(texts)]
#     tfidf_ret = TFIDFRetriever(vectorizer=vec, docs=tfidf_docs, tfidf_array=tfidf_array)

#     # reconnect to the same Weaviate index
#     dense = Weaviate(
#         client=client,
#         index_name=data["index_name"],
#         text_key=data["text_key"],
#         embedding=embeddings,
#         by_text=False 
#     )

#     return EnsembleRetriever(
#         retrievers=[tfidf_ret, dense.as_retriever()],
#         weights=data["weights"],
#     )

# # ---- build or load hybrid retriever ----
# if os.path.exists(RETRIEVER_FILE):
#     print("Loading saved retriever...")
#     retriever = load_retriever(RETRIEVER_FILE, chunked_docs)
# else:
#     print("Creating new retriever...")

#     # upsert chunks to Weaviate (server persists)
#     dense_vs.add_documents(chunked_docs)

#     # TF-IDF on the same chunked corpus (use params dict, not a prebuilt vectorizer)
#     tfidf_ret = TFIDFRetriever.from_documents(chunked_docs, tfidf_params=TFIDF_PARAMS)

#     # assemble hybrid (TF-IDF + dense)
#     retriever = EnsembleRetriever(
#         retrievers=[tfidf_ret, dense_vs.as_retriever()],
#         weights=[0.4, 0.6],
#     )
#     save_retriever(retriever, RETRIEVER_FILE)

# print("✅ Hybrid (TF-IDF + Weaviate) retriever ready.")
# print("schema classes:", [c["class"] for c in client.schema.get().get("classes", [])])

docs: 23  →  chunks: 121
Loading saved retriever...
✅ Hybrid (TF-IDF + Weaviate) retriever ready.
schema classes: ['Persian_documents_index']


In [ ]:
# query = "میتوانی کمی راجب تسهیلات فرزندآوری توضیح بدی."
# q_norm = normalize_fa(query)  # keep query normalization consistent with docs

# results = retriever.get_relevant_documents(q_norm)

# for i, r in enumerate(results, start=1):
#     meta = r.metadata or {}
#     src = meta.get("source") or meta.get("file_path") or meta.get("doc_id") or "-"
#     preview = (r.page_content or "").strip().replace("\n", " ")[:160]
#     print(f"{i:02d}. {src} — {preview} …")

## Chain

This code implements a **Retrieval-Augmented Generation (RAG)** pipeline for **Persian text processing**. The architecture combines **retrieval-based search** with **generation-based models** to answer user queries by pulling relevant information from pre-loaded Persian documents.

Key components:
- **Retriever**: Uses the **FAISS** vector store to fetch relevant Persian documents based on the user's query.
- **Prompt Template**: A **Persian-language prompt** is designed to inject the retrieved context and user question into a model, generating contextually relevant answers.
- **LLM (Large Language Model)**: The model processes the query and relevant context, generating a **natural language response** in Persian.
- **Output Parsing**: The raw output from the model is parsed into a readable, clean format for user-friendly display.

### Workflow:
1. **User Query**: The user's question is passed into the retrieval system.
2. **Context Retrieval**: The relevant document chunks are retrieved based on the query.
3. **Prompt Construction**: The retrieved context and user question are combined into a prompt.
4. **LLM Generation**: The language model generates a response based on the provided context.
5. **Output Parsing**: The response is cleaned and returned to the user.

This approach enhances the ability to answer domain-specific queries in **Persian** by leveraging document retrieval and **fine-tuned language models**.


In [19]:
import huggingface_hub
print(huggingface_hub.__version__)


0.24.5


In [46]:
from langchain_core.runnables import Runnable
import getpass
import requests

class HuggingFaceInferenceRunnable(Runnable):
    def __init__(self, model_id: str, huggingface_token: str):
        self.model_id = model_id
        self.huggingface_token = huggingface_token
        self.inference_url = "https://router.huggingface.co/hf-inference"

    def invoke(self, input_dict, config: dict = None, **kwargs) -> dict:
        # Check if the input is a ChatPromptValue, and extract the question
        if isinstance(input_dict, dict):
            query = input_dict.get("question")
        elif hasattr(input_dict, 'to_string'):
            # If it's a ChatPromptValue, convert it to string and extract the question
            query = input_dict.to_string().strip()
        else:
            raise ValueError(f"Unexpected input type: {type(input_dict)}")

        # Prepare the headers for the request
        headers = {
            "Authorization": f"Bearer {self.huggingface_token}",
        }

        # Prepare the payload (input data for the model)
        payload = {
            "inputs": query,  # The input question to the model
            "parameters": {"temperature": 0.3, "max_new_tokens": 512},
        }

        # Send a POST request to the inference endpoint
        response = requests.post(self.inference_url, headers=headers, json=payload)

        # Check for successful response
        if response.status_code == 200:
            # Extract and return the model's output
            output = response.json()
            return {"response": output}  # Return the model's response as a dictionary
        else:
            raise Exception(f"Error: {response.status_code} - {response.text}")


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from operator import itemgetter


huggingface_token = getpass.getpass("Enter your Hugging Face API token: ")

hf_inference_runnable = HuggingFaceInferenceRunnable(
    model_id="HooshvareLab/bert-fa-base-uncased",  ############### Whatever model i used i couldnt get it using the wrapper of the langchain for huggingface
    huggingface_token=huggingface_token
)

# Prompt Template
prompt = ChatPromptTemplate.from_template("""
شما یک دستیار هوشمند هستید که با استفاده از اطلاعات بازیابی‌شده از اسناد، به سوالات کاربر پاسخ می‌دهید.
در صورت عدم وجود اطلاعات کافی، بگویید که پاسخ دقیقی در اسناد پیدا نشد.

🧠 متن بازیابی‌شده:
{context}

❓ سوال کاربر:
{question}

✍️ پاسخ شما (به فارسی و روان):
""")

# Output Parser
output_parser = StrOutputParser()

# RAG Setup
RAG = RunnableParallel(
    {
        "context": itemgetter("question") | retriever_faiss,  # or 'retriever' if hybrid
        "question": RunnablePassthrough(),
    }
)

# Chain
chain = RAG | prompt | hf_inference_runnable | output_parser


In [48]:
query = "میتوانی کمی راجب تسهیلات فرزندآوری توضیح بدی؟"
result = chain.invoke({"question": query})

print("پاسخ مدل:")
print(result)


Exception: Error: 404 - Not Found